## **1. Load Data**

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel
import torch

tqdm.pandas()

c:\Workplace\Data_Analyst\VNNewsClassifcation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../Data/vietnamese_news_for_modeling.csv')

In [3]:
df.head()

,title,content,category,raw_word_count,clean_content,clean_title,token_count,word_count
0,Nha Trang có thêm dự án nhà ở xã hội hơn 1.100...,UBND tỉnh Khánh Hòa vừa ban hành quyết định ch...,Bất động sản,530,ubnd tỉnh khánh_hòa ban_hành quyết_định chấp_t...,nha_trang dự_án xã_hội tỷ đồng,214,214
1,Đề xuất Nhà nước thu hồi đất khi một nửa chủ s...,"Quan điểm này được ông Đỗ Đức Hồng Hà, Phó chủ...",Bất động sản,811,quan_điểm đỗ đức hồng_hà phó chủ_nhiệm ủy_ban ...,đề_xuất nhà_nước thu_hồi đất một_nửa chủ_sở_hữ...,276,276
2,Xu hướng dùng nội thất inox trong thiết kế khô...,"Với đặc tính chống gỉ, chịu lực, dễ vệ sinh và...",Bất động sản,911,đặc_tính chống gỉ lực vệ_sinh bề_mặt thẩm_mỹ c...,xu_hướng nội thất_inox thiết_kế không_gian hiệ...,363,363
3,Cải tạo căn hộ 48 m2 cho 3 người ở với 280 tri...,"Căn hộ có diện tích 48 m2, tọa lạc tại một tòa...",Bất động sản,869,căn_hộ diện_tích m tọa_lạc tòa chung_cư tp hcm...,cải_tạo căn_hộ m triệu đồng,379,379
4,Thu ngân sách TP HCM vượt cùng kỳ nhờ đất đai,Thông tin được Phó giám đốc Sở Tài chính TP HC...,Bất động sản,603,thông_tin phó giám_đốc sở tài_chính tp hcm hoà...,thu ngân_sách tp hcm kỳ đất_đai,237,237


In [4]:
df['clean_content'].isnull().sum()

np.int64(0)

In [5]:
df['clean_title'].isnull().sum()

np.int64(0)

In [6]:
df.dropna(subset=['clean_title'], inplace=True)

In [7]:
df = df.dropna(subset=['clean_content', 'clean_title', 'token_count', 'category']).reset_index(drop=True)

## **2. Xác Định Dữ Liệu Đầu Vào**

In [8]:
df.head(2)

,title,content,category,raw_word_count,clean_content,clean_title,token_count,word_count
0,Nha Trang có thêm dự án nhà ở xã hội hơn 1.100...,UBND tỉnh Khánh Hòa vừa ban hành quyết định ch...,Bất động sản,530,ubnd tỉnh khánh_hòa ban_hành quyết_định chấp_t...,nha_trang dự_án xã_hội tỷ đồng,214,214
1,Đề xuất Nhà nước thu hồi đất khi một nửa chủ s...,"Quan điểm này được ông Đỗ Đức Hồng Hà, Phó chủ...",Bất động sản,811,quan_điểm đỗ đức hồng_hà phó chủ_nhiệm ủy_ban ...,đề_xuất nhà_nước thu_hồi đất một_nửa chủ_sở_hữ...,276,276


In [9]:
X = df[['clean_content', 'clean_title',  'token_count']]
y = df['category']
X['clean_title'] = X['clean_title'].fillna('')

C:\Users\Admin\AppData\Local\Temp\ipykernel_13412\3544971474.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['clean_title'] = X['clean_title'].fillna('')


## **3. Chọn Embeddings**

In [10]:
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base', use_fast=True)
model = AutoModel.from_pretrained('vinai/phobert-base')

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
model.eval()

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

## **4. Chuyển Text Thành Embeddings**

In [12]:
def get_embeddings_batch(text_list, batch_size=32, max_length=256):
    """Encode một list text → embeddings (mean pooling)
    Code giống bạn nhưng an toàn, không crash.
    """
    all_embeddings = []

    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i+batch_size]

        inputs = tokenizer(
            batch,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=max_length
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        emb = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

        all_embeddings.append(emb)

        print(f"Encoded {i + len(batch)} / {len(text_list)}")

    return np.vstack(all_embeddings)

In [13]:
embedding = get_embeddings_batch([X['clean_content'][0]])
embedding[0][:10]

Encoded 1 / 1


array([-0.00193525,  0.22013772, -0.33357745, -0.28355372, -0.26727015,
        0.1718816 , -0.13817936, -0.29914102,  0.11463256, -0.1539001 ],
      dtype=float32)

## **5. Tạo Pipeline cho FE**

In [14]:
X.shape

(67150, 3)

In [15]:
title_embeddings = get_embeddings_batch(
    X['clean_title'].tolist(),
    batch_size=32,
    max_length=64
)

Encoded 32 / 67150
Encoded 64 / 67150
Encoded 96 / 67150
Encoded 128 / 67150
Encoded 160 / 67150
Encoded 192 / 67150
Encoded 224 / 67150
Encoded 256 / 67150
Encoded 288 / 67150
Encoded 320 / 67150
Encoded 352 / 67150
Encoded 384 / 67150
Encoded 416 / 67150
Encoded 448 / 67150
Encoded 480 / 67150
Encoded 512 / 67150
Encoded 544 / 67150
Encoded 576 / 67150
Encoded 608 / 67150
Encoded 640 / 67150
Encoded 672 / 67150
Encoded 704 / 67150
Encoded 736 / 67150
Encoded 768 / 67150
Encoded 800 / 67150
Encoded 832 / 67150
Encoded 864 / 67150
Encoded 896 / 67150
Encoded 928 / 67150
Encoded 960 / 67150
Encoded 992 / 67150
Encoded 1024 / 67150
Encoded 1056 / 67150
Encoded 1088 / 67150
Encoded 1120 / 67150
Encoded 1152 / 67150
Encoded 1184 / 67150
Encoded 1216 / 67150
Encoded 1248 / 67150
Encoded 1280 / 67150
Encoded 1312 / 67150
Encoded 1344 / 67150
Encoded 1376 / 67150
Encoded 1408 / 67150
Encoded 1440 / 67150
Encoded 1472 / 67150
Encoded 1504 / 67150
Encoded 1536 / 67150
Encoded 1568 / 67150
Encod

In [16]:
content_embeddings = get_embeddings_batch(
    X['clean_content'].tolist(),
    batch_size=32,
    max_length=256
)

Encoded 32 / 67150
Encoded 64 / 67150
Encoded 96 / 67150
Encoded 128 / 67150
Encoded 160 / 67150
Encoded 192 / 67150
Encoded 224 / 67150
Encoded 256 / 67150
Encoded 288 / 67150
Encoded 320 / 67150
Encoded 352 / 67150
Encoded 384 / 67150
Encoded 416 / 67150
Encoded 448 / 67150
Encoded 480 / 67150
Encoded 512 / 67150
Encoded 544 / 67150
Encoded 576 / 67150
Encoded 608 / 67150
Encoded 640 / 67150
Encoded 672 / 67150
Encoded 704 / 67150
Encoded 736 / 67150
Encoded 768 / 67150
Encoded 800 / 67150
Encoded 832 / 67150
Encoded 864 / 67150
Encoded 896 / 67150
Encoded 928 / 67150
Encoded 960 / 67150
Encoded 992 / 67150
Encoded 1024 / 67150
Encoded 1056 / 67150
Encoded 1088 / 67150
Encoded 1120 / 67150
Encoded 1152 / 67150
Encoded 1184 / 67150
Encoded 1216 / 67150
Encoded 1248 / 67150
Encoded 1280 / 67150
Encoded 1312 / 67150
Encoded 1344 / 67150
Encoded 1376 / 67150
Encoded 1408 / 67150
Encoded 1440 / 67150
Encoded 1472 / 67150
Encoded 1504 / 67150
Encoded 1536 / 67150
Encoded 1568 / 67150
Encod

In [17]:
token_count_feat = np.array([[c] for c in X['token_count']])

In [18]:
X_features = np.hstack([
    title_embeddings,
    content_embeddings,
    token_count_feat
])

## **6. Lưu Embeddings Để Tái Sử Dụng**

In [19]:
np.save('../pipelines/X_features.npy', X_features)
np.save('../pipelines/y_labels.npy', y.to_numpy())